In [234]:
import sympy as smp
import numpy as np
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from sklearn.datasets import make_classification
smp.init_printing()


In [235]:
fps = 50
t_final = 60
frames_number = fps*t_final
t_values = np.linspace(0, t_final, frames_number)

In [236]:
P = 20
N = 3
M = 2
t = smp.symbols('t', real=True, positive=True)

w = [[smp.Function(f'omega^{k}_{i}', real=True)(t) for i in range(M)] for k in range(N)]
z = [[smp.symbols(f'zeta^{u}_{i}', real=True) for i in range(M)] for u in range(P)]
x = [[smp.symbols(f'xi^{u}_{k}', real=True) for u in range(P)] for k in range(N)]


In [237]:
sum = 0
for i in range(M):
    for u in range(P):
        for k in range(N):
            sum += (z[u][i] - w[k][i] * x[k][u])**2

E = smp.Rational(1,2) * sum

In [238]:
# From neural networks to euler
q = w
V = E

In [240]:
q[0]

In [242]:
q_d = []
q_dd = []
for u in range(N):
    for i in range(M):
        q_d.append(smp.diff(q[u][i],t))
        #q_dd.append(smp.diff(q_d[u][i],t))


In [243]:
q_d

In [ ]:
T = 0
for k in range(N):
    T += q_d[k]**2
T = smp.Rational(1,2) * T

In [ ]:
L = (T - V).simplify()

In [ ]:
E_L = [smp.diff(L, q[u]) - smp.diff(smp.diff(L, q_d[u]), t).simplify() for u in range(N)]

In [ ]:
equilibrum_eq = [expr.subs([(sym, 0) for sym in q_dd]) for expr in E_L]

In [ ]:
x_data, z_data = make_classification(n_samples=P, n_features=N, n_redundant=0, n_classes=2, n_clusters_per_class=1, class_sep=10)
x_data[:, :1] = 1
z_data

In [ ]:
eq_func = [smp.lambdify((x, z, w), equilibrum_eq[u]) for u in range(N)]
eq_real = [eq_ifunc(x_data.T, z_data, w) for eq_ifunc in eq_func]
w_data = list(smp.linsolve(eq_real, tuple(w)).args[0])

In [ ]:
E_func = smp.lambdify((x,z,w), E)
E_func(x_data.T, z_data, w_data)

In [ ]:
E_func(x_data.T, z_data, np.random.random(3))

In [ ]:
def sep(x):
  return -w_data[1] / w_data[2] * x + w_data[0] / w_data[2]

x = np.linspace(np.min(x_data[:,1]), np.max(x_data[:,1]), 20)
plt.plot(x, sep(x))



color = {-1: 'red', 1:'blue'}
for i in range(P):
  plt.scatter([x_data[i,1]], [x_data[i,2]], c=color[z_data[i]])

In [ ]:
sols = {}
for acceleration, equation in zip(q_dd, E_L):
    c = smp.diff(equation, acceleration)
    solution = (acceleration - equation/c).simplify()
    sols[acceleration] = solution

sols

In [ ]:
dvdt_f = [smp.lambdify((t,p,m,q,q_d), sols[q_dd[u]]) for u in range(n)]
drdt_f = [smp.lambdify((t,p,m,q,q_d), q_d[u]) for u in range(n)]

dSdt_f = drdt_f + dvdt_f
def dSdt(S, t, p, m):
    return [dSdt_f[u](t,p,m,S[:n],S[n:]) for u in range(2*n)]

In [ ]:
ans = odeint(dSdt, y0=initial_conditions, t=t_values, args=(p_values,m_values))
q_f = ans.T[:n]

In [ ]:
r_f = smp.lambdify((t,p,q), r)

x = []
y = []
pos = r_f(t_values,p_values,q_f)
for r_i in pos:
    x_i = r_i[0]
    y_i = r_i[1]
   
    x.append(x_i)
    y.append(y_i)


In [ ]:
fig, ax = plt.subplots(dpi=1920/16)
line, = plt.plot([], [], "o", markersize=5)

scale = 5
ax.set_xlim(-scale, scale)
ax.set_ylim(-scale, scale)

textEquations = ''
diferentialNames = ''
for u in range(N):
    #diferentialName = r'$' + str(smp.latex(q_dd[u])) + '$ = ' + '\n'
    #diferentialNames += diferentialName
    textEquation = r'$' + str(smp.latex(sols[q_dd[u]])) + '$ '
    textEquation += '\n'
    textEquations += textEquation

plt.text(-scale, scale, diferentialNames)
#line.set_label(r'$' + str(smp.latex(sols[q_dd[0]])) + '$')
time_text = ax.text(-scale, scale-4, '', fontsize=15)
time_text.set_text(textEquations)
#plt.text(-5, 5, r'$' + str(smp.latex(q_dd[0])) + ' = ' + str(smp.latex(sols[q_dd[0]])) + '$',wrap=True)

def animate(frame):
    line.set_data(([0] + [y[i][0][frame] for i in range(N)], [0] + [-x[i][0][frame] for i in range(N)]))
    time_text.set_position((-scale - frame/4, scale-4))
    return line

anim = FuncAnimation(fig, animate, frames=frames_number, interval=20)
plt.axis('off')
plt.show()


In [ ]:
FFwriter = animation.FFMpegWriter(fps=fps)
anim.save('GravityPendulum.mp4',writer=FFwriter)